In [45]:
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
import GridWorld_env
from replay_buffer import ReplayBuffer
import gymnasium as gym
import random
import math
from itertools import count

device = "cuda" if torch.cuda.is_available() else "cpu"
is_ipython = "inline" in matplotlib.get_backend()
if is_ipython:
    from IPython import display
    
from collections import deque, namedtuple

Transition = namedtuple("Transition", ("state", "action", "next_state", "reward"))

In [46]:
BATCH_SIZE = 64
GAMMA = 0.99
EPS_START = 0.99
EPS_END = 0.01
EPS_DECAY = 1500000
TAU = 0.0002
STEPSIZE = 0.00005
BETA_START = 0.4
BETA_END = 1
BETA_LINEAR_CAP = 5500 * 1500
N_STEP = 1

steps_done = 1500 * 3500

In [47]:
class DoubleDueling_DQN(nn.Module):
    def __init__(self, input_dim, action_dim):
        
        super(DoubleDueling_DQN, self).__init__()
        self.conv1 = nn.Conv3d(3, 29, 3, 1, 1)
        self.conv2 = nn.Conv3d(32, 67, 3, 1, 1)
        
        self.fc1 = nn.Linear((64 + 3*2)*input_dim * input_dim * input_dim, 1024)
        
        self.lstmcell = nn.LSTMCell(1024, 1024)
        
        self.actions = nn.Linear(1024, action_dim)
        self.advantage = nn.Linear(1024, 1)
    def forward(self, x, h, c):
        original_state = x
                    
        x = self.conv1(x)
        x = torch.cat([x, original_state], dim=1)
        x = F.relu6(x)
        
        x = self.conv2(x)
        x = torch.cat([x, original_state], dim=1)
        x = F.relu6(x)
        
        x = nn.Flatten()(x)
        x = self.fc1(x)
        x = F.relu6(x)
        
        h, c = self.lstmcell(x, (h, c))
        
        actions = self.actions(h)
        
        x = self.advantage(h) + (actions - actions.mean(dim=1, keepdim=True))
        return x, h, c


In [48]:
n_actions = 10
env = gym.make("GridWorld_env/GridWorld", dimension_size=4, path="targets")
env.reset()

policy_net = DoubleDueling_DQN(4, n_actions).to(device)


In [49]:
policy_net = torch.load("checkpoint3500_policy.pt")



In [50]:
def select_action(state, h, c, greedy=False):
    #global steps_done
    
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    #steps_done += 1
    
    if greedy:
        with torch.no_grad():
            out, h, c = policy_net(state, h, c)
            print(out)
            return out.max(1).indices.view(1,1), h, c
            #return policy_net(state, h, c).max(1).indices.view(1,1), h, c
        
    if sample > eps_threshold:
        with torch.no_grad():
            out, h, c = policy_net(state, h, c)
            print(out)
            
            return out.max(1).indices.view(1,1), h, c
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long), h, c

    

In [51]:

state, info = env.reset()
state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
h = torch.zeros(1, 1024, dtype=torch.float32, device=device)
c = torch.zeros(1, 1024, dtype=torch.float32, device=device)

for t in count():
    
    action, h, c = select_action(state, h, c, False)
    observation, reward, terminated, truncated, _ = env.step((action.item(), 0))
    print(action.item())
    reward = torch.tensor([reward], device=device)
    done = terminated or truncated

    if terminated:
        next_state = None
    else:
        next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

    state = next_state
    if terminated or truncated:
        break
    env.unwrapped.render()
    #sleep(2)
    display.clear_output(wait=True)

    


tensor([[-93.7090, -93.7293, -93.7521, -93.7482, -93.7561, -93.7305, -97.7447,
         -93.7579, -97.7213, -97.7510]], device='cuda:0')
0


KeyboardInterrupt: 